In [3]:
import numpy as np
import pickle
from tqdm import tqdm_notebook
import time
import random
import sys

In [4]:
def corrected_mean(matrix):
    
    if not np.isclose(np.count_nonzero(matrix), 0):
        return np.sum(matrix)/np.count_nonzero(matrix)
    else:
        return 2.5

In [5]:
def normalize_ratings(ratingsMatrix):
    
    normalizedRatingsMatrix = ratingsMatrix.copy()
    
    for i, userRatings in enumerate(ratingsMatrix):
        
        if np.count_nonzero(userRatings) == 0:
            residue = corrected_mean(ratingsMatrix)
        else:
            residue = np.sum(userRatings)/np.count_nonzero(userRatings)
        
        
        for j,rating in enumerate(userRatings):
            if rating != 0:
                normalizedRatingsMatrix[i][j] -= residue
            if normalizedRatingsMatrix[i][j] == 0:
                normalizedRatingsMatrix[i][j] = 1e-8
        
    return normalizedRatingsMatrix

In [6]:
def pearson_coeff(x, y):
    
    xMean = np.corrected_mean(x)
    yMean = np.corrected_mean(y)
    
    return np.sum((x - xMean)*(y - yMean)) / ((np.sum((x - xMean)**2))*(np.sum((y - yMean)**2)))

In [7]:
def cosine_dist(x, y):
    
    if np.isclose(np.dot(x,y), 0):
        return 0.0
    
    return np.dot(x, y)/(np.linalg.norm(x)*np.linalg.norm(y))

In [8]:
def get_similar_users(ratingsMatrix, userIdx, k):
    
    similarUsers = []
    thisUser = ratingsMatrix[userIdx].copy()
    
    for i, userRatings in (enumerate(ratingsMatrix)):
        
        if i == userIdx:
            continue
        
        # similarity = pearson_coeff(userRatings, thisUser)
        similarity = cosine_dist(userRatings, thisUser)
        
        similarUsers.append((i, similarity))
        
    similarUsers = sorted(similarUsers, key = lambda x: x[1], reverse = True)
    
    return similarUsers[:k]

In [9]:
def predict(ratingsMatrix, normalizedRatingsMatrix, userIdx, itemIdx, k, baseline = False):
    
    # k = 10
    similarUsers = get_similar_users(normalizedRatingsMatrix, userIdx, k)
    
    predictedRating = 0.0
    normalizationFactor = 0.0
    mu = corrected_mean(ratingsMatrix)
    cnt = 0
    b = 0.0
    
    for (user, similarity) in similarUsers:
        
        if ratingsMatrix[user][itemIdx] > 0 and similarity > 0: 
            
            # check with and without similarity > 0
            
            # user's rating should be considered only if they have rated this item
            # otherwise the missing value (if considered as zero) may cause problems
            # it can be imputed by taking the user's mean rating or item's mean rating, etc.
                
            if baseline == True:
                b = corrected_mean(ratingsMatrix[user,:])
            
            predictedRating += (ratingsMatrix[user][itemIdx] - b) * similarity
            normalizationFactor += similarity

    if not np.isclose(normalizationFactor, 0):
        predictedRating /= normalizationFactor
    else:
        return max(1, min(corrected_mean(ratingsMatrix[userIdx,:]) + corrected_mean(ratingsMatrix[:,itemIdx]) - mu, 5.0))
    
    if baseline == True:
        b = corrected_mean(ratingsMatrix[userIdx,:])
        predictedRating += b
    
    return max(1, min(predictedRating, 5.0))

In [10]:
def train_test_split(ratingsMatrix, cnt = 1000):
    
    (rows, cols) = ratingsMatrix.shape
    
    completeSet = []
    for i in (range(rows)):
        for j in range(cols):
            if ratingsMatrix[i][j] != 0:
                completeSet.append((i,j))
    random.shuffle(completeSet)
    
    testSet = random.sample(completeSet, cnt)
    trainSet = list(set(completeSet) - set(testSet))
    
    return trainSet, testSet

In [13]:
def evaluate(ratingsMatrix, normalizedRatingsMatrix, testSet, originalMatrix, k, baseline = False):
    
    N = len(testSet)
    rmse = 0
    mae = 0
    rmseB = 0
    maeB = 0
    
    for i in tqdm_notebook(range(N)):
        
        predicted = predict(ratingsMatrix, normalizedRatingsMatrix, testSet[i][0], testSet[i][1], k)
        baselinePredicted = predict(ratingsMatrix, normalizedRatingsMatrix, testSet[i][0], testSet[i][1], k, True)
        actualValue = originalMatrix[testSet[i][0]][testSet[i][1]]
        
        # print(i)
        # print('Predicted: ', predicted)
        # print('Predicted (with baseline): ',baselinePredicted)
        # print('Actual: ', actualValue)
        # print()
        
        bError = abs(actualValue - baselinePredicted)
        error = abs(actualValue - predicted)
        
        rmse += error**2
        rmseB += bError**2
        mae += error
        maeB += bError
        
    rmse /= N
    rmseB /= N
    mae /= N
    maeB /= N
    
    return mae, maeB, rmse, rmseB

In [15]:
if __name__ == '__main__':
    
    # ratingsMatrix = np.array([[4,0,0,5,1,0,0],[5,5,4,0,0,0,0],[0,0,0,2,4,5,0],[0,3,0,0,0,0,3]], dtype = float)
    # ratingsMatrix = [[4, 0, 0, 5],[5, 5, 4, 0],[0, 0, 0, 2],[0, 3, 0, 0],[1, 2, 0, 1],[0, 4, 3, 1]]
    # textMatrix = np.array([[4,0,0,5,1,0,0],[5,5,4,0,0,0,0],[0,0,0,2,4,5,0],[0,3,0,0,0,0,3]], dtype = float)
    # normalized = normalize_ratings(textMatrix)
    # print(predict(textMatrix, normalized, 0, 0, True))

    random.seed(1)

    with open('ratingsMatrix_noZeros.pickle', 'rb') as file:
        ratingsMatrix = pickle.load(file)
    ratingsMatrix = np.array(ratingsMatrix, dtype = float)

    originalMatrix = ratingsMatrix.copy()
    trainSet, testSet = train_test_split(originalMatrix, 100) #10s

    # < 1s
    for index in testSet:
        ratingsMatrix[index[0]][index[1]] = 0.0

    normalizedRatingsMatrix = normalize_ratings(ratingsMatrix)   # 18s
    
    k = 10
    mae, maeB, rmse, rmseB = evaluate(ratingsMatrix, normalizedRatingsMatrix, testSet, originalMatrix, k)
    
    print(mae, maeB, rmse, rmseB)

0.7332229591390438 0.653945381833294 0.8997287069437572 0.7658410654834132
